# [TF-IDF](https://zh.wikipedia.org/wiki/TF-IDF)

覺得 TF-IDF 的維基百科寫的滿好的, 因為很難得有維基百科的內容我都看得懂的, 哈哈

TF-IDF是一種統計方法，用以評估一字詞對於一個 **文件集** 或一個 **語料庫** 中的 **其中一份文件** 的 **重要程度**。

維基百科其實給了一個很繞舌的定義, 假設我們先簡化問題

如何評估一字詞對於一份文件的重要程度?

很簡單的想法就是看這個字詞的 **出現次數**, **出現越多次**, **越重要**, 舉例來說, 隨便挑一篇 [網路新聞](http://www.nownews.com/n/2016/11/29/2322028)

In [12]:
news1 = u'''
今年上半三星推出S7系列，獲得相當好評，下半年Note7系列原本眾所矚目，但電池事件讓商譽大受影響，外傳三星希望藉由明年S8重新振作，可能2月就會發表，將會有5.7吋與6.8吋版本，並且採用極窄邊框設計。

小米MIX將聽筒與Home鍵取消，加上極窄邊框設計，螢幕比率高達91.3％，外傳三星S8可能也會有類似設計，而且螢幕比率可能更高，不論5.7吋或6.2吋版本，都會搭配雙曲面螢幕，讓畫面幾乎看不出邊框。

另一方面，據說三星也會「參考」蘋果部分設計，例如3D Touch，並且採用雙鏡頭設計，加強語音協助功能，三星找上Viv公司協助，這間公司曾經參與Siri設計，部分工作人員曾經參與Siri初期研發，可能讓大幅強化三星語音功能。

另一方面，對於三星恐怕是另一項利多，美國即將上任總統川普，不斷要求蘋果將生產線移回美國，並且否決TPP與北美貿易協定，假使iPhone零件國外生產，僅在美國組裝，售價約增加5％，完全美國製造iPhone 7 Plus 256GB售價恐怕由969美元大幅增加至2000美元，約新台幣63000元。
'''

- - -

## TF (Term Frequency)

斷詞之後計算字詞出現的次數

In [13]:
import jieba
from collections import Counter
c = Counter(jieba.lcut(news1))
print repr(c.most_common(10)).decode('unicode-escape')

[(u'，', 31), (u'
', 8), (u'三星', 7), (u'設計', 6), (u'與', 5), (u'。', 4), (u' ', 4), (u'吋', 4), (u'可能', 4), (u'會', 4)]


假設我 **先不管標點符號跟換行** 那些問題, **三星** 是出現最多次的詞, 我可以說 **三星** 重要程度最高

考慮一個 **極端** 的例子, 有另一篇新聞內容是 **三星三星三星三星三星三星**, 三星出現 6 次, 所以, 三星的重要程度在這一篇新聞低於上一篇新聞?

事實上, 這篇新聞三星的重要程度是高於前一篇的, 因為他除了三星以外沒有提到別的字

為了避免文章長短不一的情形, 會把字詞的次數除以總詞數, 就形成 TF-IDF 的 **TF**

![Certificate](https://wikimedia.org/api/rest_v1/media/math/render/svg/05f25eceb713717766dd0b8ef4fd7d4a2f1a7a30)

介紹個工具 [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit)

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
def tokenize(text):
    return text.split()
vec = CountVectorizer(tokenizer=tokenize)
data = vec.fit_transform([' '.join(jieba.lcut(news1))]).toarray()
vocab = vec.get_feature_names()
print repr(dict(sorted(zip(vocab, data[0]), key=lambda x: -x[1])[:10])).decode('unicode-escape')

{u'會': 4, u'。': 4, u'並且': 3, u'三星': 7, u'與': 5, u'吋': 4, u'，': 31, u'設計': 6, u'螢幕': 3, u'可能': 4}


* `tokenizer` 是可以自己寫的, `token_pattern` 可以寫正則

* **竟然沒有除以總詞數的參數可以調, 太令我失望了**

* **其實有喇**, 不過要用 [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html), 搞得有點複雜, `use_idf=False` 表示不考慮 idf, 即只有 TF

In [90]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(data)
data = tf_transformer.transform(data).toarray()
print repr(dict(sorted(zip(vocab, data[0]), key=lambda x: -x[1])[:10])).decode('unicode-escape')

{u'會': 0.10705754551443787, u'。': 0.10705754551443787, u'並且': 0.080293159135828407, u'三星': 0.18735070465026626, u'與': 0.13382193189304736, u'吋': 0.10705754551443787, u'，': 0.82969597773689352, u'設計': 0.16058631827165681, u'螢幕': 0.080293159135828407, u'可能': 0.10705754551443787}


- - -

## IDF (inverse document frequency)

其實我前面逃避了一些問題, 比如標點符號, 與或可能, 這些常常出現但卻一點都不重要的字詞

當然, 標點符號可以透過前處理先去掉, 常用詞可以透過 stopwords 去掉

不過, 這裡我提供一些不同的想法

**逗點出現很多次, 但是因為每篇新聞都會有逗點, 所以, 逗點不重要**

就好像你有兩隻眼睛沒什麼了不起, 如果你有三隻眼睛, 那就真的是天賦異稟了

所以, **IDF 就是用來衡量一個詞語普遍重要性的度量**

公式有點複雜, 分母是這個**字詞出現在語料庫的文件數**, 分子是**語料庫文件的總數**

![Certificate](https://wikimedia.org/api/rest_v1/media/math/render/svg/732293caf31dae6e14c0aea986bf4a951187796b)

所以, 以 common sense 來看, 如果語料庫有 10 篇新聞, 肯定 10 篇都有逗點, 所以, 逗點的 idf 就是 `log(10/10) = log(1) = 0`, 逗點就變得一點都不重要了

- - -

## TFIDF

聽起來 TF 跟 IDF 都有對字詞的重要性有貢獻, TFIDF 就只是很單純的把他們乘在一起而已

![Certificate](https://wikimedia.org/api/rest_v1/media/math/render/svg/252332a1e2cbaa57b804bb09e6a3690358e26927)

當然 tfidf 需要有語料庫, 隨便多挑幾篇新聞

In [26]:
news2=u'''
國家通訊委員會（NCC）決議，未來智慧型手機經由電信商綁約，或政府部門大量購買，都將「建議」具備資安認證，內建作業系統與應用程式都須檢查，最慢預計明（106）年中實施，消費者資安風險可望降低。

NCC發言人翁柏宗表示，各國政府除大陸外，不會強制手機必須檢查資安，因此採用建議方式，希望電信商販賣或政府統一採購手機，內建作業系統與應用程式，都能通過資訊安全實驗室認證，但民眾自行下載應用程式則未管制。

境外手機廠商是否遵照「建議」；翁柏宗說明，將提出修訂政府採購相關規定，未來購買具有檢測證明手機，另外消費者若能自主要求，電信業者就會轉向手機製造商轉達意見，日前蘋果、三星、HTC都有參與相關會議。

未來手機資安檢測區分三級，初級針對個人隱私與資料保護，中級除個人資料外，資料檔案儲存與傳輸也要能夠維護安全，高等標準則專注手機系統安全，最底層運作程式不能遭受竄改，不論iOS或Android。

執行安全測試實驗室須經過ISO認證，相關細部規範仍在規劃，未來必須展開預告與意見諮詢等階段，需要3個月時間，也要等待實驗室核可認證，因此NCC預估最慢明年中能夠上路實施。
'''

In [30]:
news3=u'''
部分iPhone 6s電量用到一半就會自動關機，而且要插上電源才能重新開機，全球使用者與蘋果反應，最後才有免費換電池方案，但是否與電池安全性有關，始終無法得知，但日前蘋果中國官方網站發出聲明，簡述問題來由。

聲明中表示，少數於2015年9月到10月期間生產的iPhone 6s，某項電池元件被裝進電池包之前，在受控環境空氣中暴露時間過長，使電池電量消耗速度快於正常水準，導致意外關機；同時強調是不是安全問題。

中國蘋果特別提到，iPhone在設計時有意允許設備在一定條件下自動關機，如極度寒冷的氣溫，發生自動關機對於顧客而言可能是意外，但這種設計是為了保護電子元件避免由於低電壓遭到損傷。

然而台灣蘋果官網僅公告，已確定有極少數iPhone 6s裝置可能會意外關機。這並非安全問題，它只會影響序號在有限範圍內並於2015年9月至10月間製造的裝置；想要跟換電池使用者，盡快到官網輸入序號查詢是否符合資格。
'''

In [33]:
news4=u'''
任何新款手機一旦傳出訊號差或續行力不好，不論效能多好都會被消費者打上問號，恐怕嚴重影響銷量，芬蘭阿爾托大學研發出懸新天線，能夠強化訊號接收與發射能力，電池損耗也能降低，非常具有未來發展前景。

目前智慧型手機暗藏許多天線，一個天線對應一個或幾個頻段，例如蜂巢式天線、GPS、藍牙，新型天線拋棄這項設計，將幾個小天線組成一個獨立天線器統，藉由數字控制調整接收頻段，讓天線數量能夠減少。

阿爾托大學教授Ville Viikar表示，目前手機可能約有7個天線，利用新研發技術能夠減少至2個，因為系統能夠調整適當接收頻段，因此手機就能多出空間，減少邊框營不加大，而且不會犧牲手機性能，手機效能大幅提昇。

Ville Viikar強調，智慧型手機流行趨勢，就是要能擴大螢幕，但很難捨去最上下兩端，因為必須放置許多天線，接收行動訊號、WIFI、藍牙等訊號，並且新型天線可能延長電池壽命，但仍需要更多實驗數據佐證。
'''

In [35]:
news5=u'''
任何新款手機一旦傳出訊號差或續行力不好，不論效能多好都會被消費者打上問號，恐怕嚴重影響銷量，芬蘭阿爾托大學研發出懸新天線，能夠強化訊號接收與發射能力，電池損耗也能降低，非常具有未來發展前景。

目前智慧型手機暗藏許多天線，一個天線對應一個或幾個頻段，例如蜂巢式天線、GPS、藍牙，新型天線拋棄這項設計，將幾個小天線組成一個獨立天線器統，藉由數字控制調整接收頻段，讓天線數量能夠減少。

阿爾托大學教授Ville Viikar表示，目前手機可能約有7個天線，利用新研發技術能夠減少至2個，因為系統能夠調整適當接收頻段，因此手機就能多出空間，減少邊框營不加大，而且不會犧牲手機性能，手機效能大幅提昇。

Ville Viikar強調，智慧型手機流行趨勢，就是要能擴大螢幕，但很難捨去最上下兩端，因為必須放置許多天線，接收行動訊號、WIFI、藍牙等訊號，並且新型天線可能延長電池壽命，但仍需要更多實驗數據佐證。'''

In [37]:
news = [' '.join(jieba.lcut(news1)), ' '.join(jieba.lcut(news2)), ' '.join(jieba.lcut(news3)), ' '.join(jieba.lcut(news4)), 
        ' '.join(jieba.lcut(news5))]

### idf

In [91]:
vec = CountVectorizer(tokenizer=tokenize)
data = vec.fit_transform(news).toarray()
vocab = vec.get_feature_names()

這邊要設定 `use_idf=True`

In [104]:
tf_transformer = TfidfTransformer(use_idf=True).fit(data)
data = tf_transformer.transform(data).toarray()

把 idf 結果存成 `dict`

In [105]:
idf = vectorizer.idf_
result_dict = dict(zip(vectorizer.get_feature_names(), idf))

In [106]:
result_dict[u'，']

1.0

** ??????????????????????????? **

*，* 的 idf 應該要是 0 阿, 搞笑了

根據維基百科的說法, 計算 idf 時, 為了避免該詞語不在語料庫中，就會 **導致分母為零**，因此一般情況下使用

![123](https://wikimedia.org/api/rest_v1/media/math/render/svg/7686335bc9835f6f3c5fb6403e9857bc7f0aa7e7)

但是, 根據 sklearn 文件 [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html), idf 是這樣算的 **`idf(d, t) = log [ (1 + n) / 1 + df(d, t) ] + 1`**, 這跟維基的定義是不太一樣的

不過, 我 **覺得** sklearn 的算法可能是比較合理的, 因為如果只有分母加一, 以逗點的例子, idf 變成 log(5/6) 就小於 0 了, 另外, sklearn 的 idf 最小就是 1, 這在維基也沒有特別提到

### TFIDF

In [109]:
print repr(sorted(zip(vectorizer.get_feature_names(), data[0]), key=lambda x: -x[1])[:10]).decode('unicode-escape')

[(u'吋', 0.28318600800486154), (u'邊框', 0.2123895060036462), (u'三星', 0.19112247982429206), (u'5.7', 0.14159300400243077), (u's8', 0.14159300400243077), (u'siri', 0.14159300400243077), (u'公司', 0.14159300400243077), (u'功能', 0.14159300400243077), (u'協助', 0.14159300400243077), (u'另一方面', 0.14159300400243077)]


姑且不論效果或 idf 的計算方式, 逗點本來是出現次數最多的, 但是透過 tfidf, 連前 10 名都進不去了

- - -

其實, sklearn 還有提供很多功能, 這裡稍微介紹一下其他 `arg`

### `CountVectorizer`

* `ngram_range`:  (1, n), 考慮所有 N-gram N<=n

* `binary`: 不考慮次數, 有出現 1, 沒出現 0

### `TfidfTransformer`

* `smooth_idf`: 兩種不同的 idf 計算方式

* `sublinear_tf`: replace tf with 1 + log(tf).

### `TfidfVectorizer`

Equivalent to CountVectorizer followed by TfidfTransformer.

- - -

* tfidf 最常見的應用在於 **關鍵字** 選取

* 大量語料庫, 效果會更好

* 再回頭來看那繞舌的定義

* TF-IDF是一種統計方法，用以評估一字詞對於一個 **文件集** 或一個 **語料庫** 中的 **其中一份文件** 的 **重要程度**。

* **一字詞對於一個文件集或一個語料庫中** (idf 需要跨文本比較)

* **的其中一份文件** (一字詞的 tf 在每份文件都 **不一定一樣**, 所以只能針對其中一份文件)

* **一個字詞的 idf 是一樣的, 但是每篇的 tf 不一樣, 所以, 每個字詞在不同文件的 tfidf 都是不一樣的**

* 也因為如此 tfidf 對每篇文章都具有 **獨特的代表性**, 所以, 常常會利用來做後續的機器學習或分析

* TF-IDF權重計算方法經常會和**餘弦相似性**（cosine similarity）一同使用於向量空間模型中，用以判斷兩份文件之間的**相似性**

* sklearn 提供了很多 **分群** 的 [範例](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#examples-using-sklearn-feature-extraction-text-tfidfvectorizer)

* 維基最後面有提到 tfidf 的缺點

* **也許可以透過 idf 來維護 stopwords**